<a href="https://colab.research.google.com/github/Kushagrasaxena11/Amazon_reviews_analysis/blob/main/Amazon_reviews_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.optimizers import SGD, Adam
import bz2
import csv
from sklearn.metrics import roc_auc_score

In [4]:
pip install opendatasets

In [5]:
import opendatasets as od

In [6]:
od.download("https://www.kaggle.com/datasets/bittlingmayer/amazonreviews")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: kushagrasaxena95
Your Kaggle Key: ··········


100%|██████████| 493M/493M [00:05<00:00, 87.5MB/s]


In [12]:
# Load the training data
training_data = bz2.BZ2File("/content/amazonreviews/train.ft.txt.bz2")
training_data = training_data.readlines()
training_data = [x.decode('utf-8') for x in training_data]
print(len(training_data))

# Load the test data
test_data = bz2.BZ2File("/content/amazonreviews/test.ft.txt.bz2")
test_data = test_data.readlines()
test_data = [x.decode('utf-8') for x in test_data]
print(len(test_data))

3600000
400000


In [13]:
# Split the data into labels and texts
train_labels = [int(re.findall(r'__label__(\d)', line)[0]) for line in training_data]
train_texts = [re.sub(r'__label__\d ', '', line) for line in training_data]

test_labels = [int(re.findall(r'__label__(\d)', line)[0]) for line in test_data]
test_texts = [re.sub(r'__label__\d ', '', line) for line in test_data]

# Convert labels to binary (0 and 1)
train_labels = [0 if label == 1 else 1 for label in train_labels]
test_labels = [0 if label == 1 else 1 for label in test_labels]

In [14]:
def clean_text(text):
    # Remove non-alphanumeric characters and extra whitespace
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convert multiple whitespace characters to a single space
    text = re.sub(r'\s+', ' ', text)
    # Convert the text to lowercase
    text = text.lower()
    return text

In [15]:
train_texts=pd.DataFrame(train_texts)[0].apply(clean_text)
test_texts=pd.DataFrame(test_texts)[0].apply(clean_text)

In [16]:
pd.DataFrame(test_labels).value_counts()

0    200000
1    200000
dtype: int64

In [17]:
len(train_texts)

3600000

In [18]:
len(test_texts)

400000

In [20]:
# Tokenization and padding
max_words = 1000
max_sequence_length = 100

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train_texts)

X_train = tokenizer.texts_to_sequences(train_texts)
X_test = tokenizer.texts_to_sequences(test_texts)

X_train = pad_sequences(X_train, maxlen=max_sequence_length)
X_test = pad_sequences(X_test, maxlen=max_sequence_length)

In [21]:
X_train = np.array(X_train)
print(X_train.shape)
X_test = np.array(X_test)
print(X_test.shape)
y_train = np.array(train_labels)
print(y_train.shape)
y_test = np.array(test_labels)
print(y_test.shape)

(3600000, 100)
(400000, 100)
(3600000,)
(400000,)


In [22]:
## Create the model

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_sequence_length))
model.add(LSTM(units=128, return_sequences=True))
model.add(Dropout(rate=0.2))
model.add(LSTM(units=128))
model.add(Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 128)          128000    
                                                                 
 lstm (LSTM)                 (None, 100, 128)          131584    
                                                                 
 dropout (Dropout)           (None, 100, 128)          0         
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 391297 (1.49 MB)
Trainable params: 391297 (1.49 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=2, batch_size=2048, verbose=1, validation_split=.1)

In [ ]:
loss,accuracy = model.evaluate(X_test, y_test)
print("Loss:", loss)
print("Accuracy:", accuracy)

In [ ]:
prediction=model.predict(X_test)
y_pred=np.where(prediction>=.5,1,0)
df=pd.DataFrame()
df['actual'],df['pred']=y_test,y_pred

In [ ]:
df

In [ ]:
# Evaluate with a confusion matrix and classification report

import seaborn as sns
CM = confusion_matrix(y_test, y_pred)
sns.heatmap(CM,annot=True,center = True,fmt='g',cmap='Blues')
CM

In [ ]:
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)